# Week 7: Machine Learning

This week we discussed the fundamentals of machine learning and reviewed some common ML algorithms. This notebook will give you more hands-on experience with programming these alogrithms for use in your own research. The goals of this notebook are:

+ Explore convex and nonconvex optimization problems.
+ Use the *scikit-learn* library to perform common ML analyses.
+ Introduce the *Tensorflow* library for constructing and training artificial neural networks.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import datasets

import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# set some style options
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.aspect'] = 'auto'
mpl.rcParams['image.cmap'] = 'jet'

## Optimization

Let's look at a very simple optimization problem where we are trying to estimate a model of the form:

$y = bx+e$

where,

+ $y$: dependent variable
+ $x$: independent variable
+ $b$: model weight to be estimated
+ $e$: measurement error

In class we saw that a common approach to obtaining a weight estimate $\hat{b}$ is to minimize the mean squared error:

$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i-\hat{y}_i)^2$

with $\hat{y_i} = \hat{b}x_i$.

For this exercise, we will be minimizing this error function via gradient descent. Since this is a univariate function, all we need to do is calculate the derivative of MSE with respect to $\hat{b}$, which is given by:

$\frac{\partial MSE}{\partial \hat{b}} = -\frac{2}{n} \sum_{i=1}^{n}(y_i - \hat{y}_i)(x_i)$



First, let's simulate some data. We will be using Gaussian noise for our measurement error.

In [ ]:
x = np.random.uniform(0,20,100)
b = 2
e = np.random.normal(0,10,len(x))
y = b*x+e
plt.plot(x,y,'o')
plt.plot(x,b*x,color = 'black',label = "True Line")
plt.legend()
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Regression Data")

This is a strictly convex optimization problem, which means there is only one minima and it is the global one. We can see this by plotting the MSE for several possible values of $\hat{b}$ (the error landscape). Our job is to find the $\hat{b}$ that results in the minimum MSE.

In [ ]:
def MSE(x,y,b_hat):
    n = len(x)
    y_hat = b_hat*x
    return((1/n)*np.sum((y-y_hat)**2))

In [ ]:
b_hats = np.arange(-10,10,.1) #candidate values of b_hat
errors = [] #array to store MSE values for each b_hat

for b_hat in b_hats:
    errors.append(MSE(x,y,b_hat))

plt.plot(b_hats,errors) #plot MSE for each b_hat
plt.plot(b,MSE(x,y,b),'o',markersize = 10,color = "black") #True value of b
plt.xlabel(r"$\hat{b}$")
plt.ylabel(r"$MSE(\hat{b})$")
plt.title("Error Landscape")

### Exercise 1
We will perform gradient descent to obtain an estimate $\hat{b}$. You will need to specify a precision value, while the learning rate $\eta$ will be given. If the difference between successive $\hat{b}$'s is less than the precision value, the gradient descent alogrithm has converged to a final estimate. 

**A.** Run the gradient descent alogrithm twice. For the first estimate, use a precision value of $.0001$ to obtain an estimate of $\hat{b}$. For the second estimate, use a smaller precision value (you choose). Plot and label the first estimated model on the left, the second estimated in the middle, and the final values of $\hat{b}$ for each estimate on the error landscape on the right.

In [ ]:
def MSE_gradient(x,y,b_hat):
    #Calculates the gradient of MSE for candidate b_hat
    n = len(x)
    y_hat = b_hat*x
    return((-2/n)*np.sum((y-y_hat)*x))

def MSE_GD(eta,precision_value):
    #Performs gradient descent of the MSE
    b_hat = np.random.normal(0,1,1) #initial guess
    converged = False
    while not converged:
        b_hat_new = b_hat-eta*MSE_gradient(x,y,b_hat)
        if np.abs(b_hat_new-b_hat)<precision_value:
            converged = True
        b_hat = b_hat_new
    return(b_hat)

In [ ]:
#Call gradient descent function to get b_hat estimate

eta = .000001 #don't change
precision_value = .0001
b_hat = MSE_GD(eta,precision_value)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(15, 4),constrained_layout=True)

#First Plot
axes[0].set_title("Model Estimate 1")
axes[0].plot(x,y,'o')
axes[0].plot(x,b*x,color = 'black',label = "True Line")
axes[0].plot(x,b_hat*x,color='red',label='Estimated Line')
axes[0].legend()

#Second Plot
axes[1].set_title("Model Estimate 2")
axes[1].plot(x,y,'o')
axes[1].plot(x,b*x, color = 'black',label = "True Line")
axes[1].plot(x,b_hat*x,color='goldenrod',label='Estimated Line')
axes[1].legend()

#Third Plot
axes[2].set_title("Error Landscape")
axes[2].plot(b_hats,errors)
axes[2].plot(b,MSE(x,y,b),'o',markersize = 5,color = "black",label = "True b")
axes[2].plot(MSE(x,y,b),'o',markersize = 5,color = "red",label = "Estimate 1")
axes[2].plot(b,'o',markersize = 5,color = "goldenrod",label = "Estimate 2")
axes[2].set_xlabel(r"$\hat{b}$")
axes[2].set_ylabel(r"$MSE(\hat{b})$")
axes[2].legend()

Let's now try a nonconvex optimization problem. Suppose we are trying to estimate a model of the form:

$y = sin(x^b)$

where,
+ $y$: dependent variable
+ $x$: independent variable
+ $b$: model weight to be estimated

In [ ]:
x = np.arange(0.001,10,.1)
b = 1.3
y = np.sin(x**b)
plt.plot(x,y)
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Sinusoidal Data")

Even though we have no measurement error, we will see that this is a challenging optimization problem when using gradient descent.

We will once again minimize the MSE in order to estimate the model weight. The derivative of the MSE is a little more complicated now:

$\frac{\partial MSE}{\partial \hat{b}} = -\frac{2}{n} \sum_{i=1}^{n}(y_i-\hat{y}_i)cos(x_i^{\hat{b}})ln(x_i)x_i^{\hat{b}}$

Below are two functions that calculate the MSE and the partial derivative of the MSE with respect to $\hat{b}$.

In [ ]:
def MSE(x,y,b_hat):
    n = len(x)
    y_hat = np.sin(x**(b_hat))
    return((1/n)*np.sum((y-y_hat)**2))

def MSE_gradient(x,y,b_hat):
    n = len(x)
    y_hat = np.sin(x**b_hat)
    return((-2/n)*np.sum((y-y_hat)*np.cos(x**b_hat)*np.log(x)*(x**b_hat)))

### Exercise 2
**A.** Plot the error landscape and put a marker at the true value of $b$. What issues do you foresee when implementing gradient descent on this function?

**B.** Perform gradient descent 3 times to find $\hat{b}$. Each time use a $\eta$ and precision value of $0.000001$. Plot the error landscape and three estimates of $\hat{b}$ on the right, and the three estimated functions on the left.

**C.** Why do you think gradient descent performed so poorly on this problem? Did you notice any effects of how you chose your random initial $\hat{b}$ on alogrithmic performance?

## Linear Regression via OLS

In Exercise 1, we solved a basic linear regression problem with gradient descent. However, the most common method of estimating regression coefficients is with the ordinary least squares (OLS) algorithm. The *scikit-learn* library has many useful functions for machine learning, including fitting linear models. Let's simulate some data and find the model coefficients via OLS estimation.

In [ ]:
x = np.random.uniform(0,20,100)
b0 = 3 #intercept
b1 = 2 #slope
e = np.random.normal(0,10,len(x))
y = b0+b1*x+e
plt.plot(x,y,'o')
plt.plot(x,b0+b1*x,color = 'black') #True regression line
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Regression Data")

In [ ]:
X = x.reshape((-1,1)) #when only one independent variable, sklearn requires this reshaping
model = LinearRegression().fit(X, y) #call linear regression from sklearn
b0_hat = model.intercept_ #estimated intercept
b1_hat = model.coef_[0] #estimated slope

In [ ]:
print("Estimate of b0:",b0_hat,"\nActual b0:",b0)
print("\nEstimate of b1:",b1_hat,"\nActual b1:",b1)

y_hat = b0_hat+b1_hat*x

plt.plot(x,y,'o')
plt.plot(x,b0+b1*x,color = 'black',label = "True Line")
plt.plot(x,y_hat,color = 'red', label = "Estimated Line")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Regression Data")
plt.legend()

We can obtain predictions of our model using the **model.predict()** function. This function will produce predicted $\hat{y}$'s for model inputs using the estimated regression coefficients. Using these predictions, we can calculate fit statistics of our model. 

$R^2$ is the proportion of variance in our dependent variable that can be predicted by our independent variables. We want our $R^2$ to be as close to 1 as possible. Different fields have different criteria for what a "good" $R^2$ is. There are several reasons the $R^2$ of a linear regression could be low. These include:

+ A linear model was not appropriate.
+ The independent variables were not good predictors of our independent variable.
+ The is large amount of measurement error.
+ OLS assumptions were violated.


In [ ]:
y_hat = model.predict(X)
mse = mean_squared_error(y,y_hat)
r2 = r2_score(y,y_hat)
print("Model MSE:",mse,"\nModel R^2:",r2)

*Scikit-learn* comes with datasets commonly used for practicing machine learning and statistical methods. We will run a multiple regression (i.e. regression with multiple independent variables) using the diabetes dataset, where we will predict disease progression.

Below, we will load the data and use the following features:

+ $y$: disease progression in patient (low good, high bad)
+ $age$: age of patient
+ $bmi$: body mass index of patient
+ $bp$: patient blood pressure

In [ ]:
x,y = sk.datasets.load_diabetes(return_X_y=True)
age = x[:,0]
bmi = x[:,2]
bp = x[:,3]
X = np.array([age,bmi,bp]).T

## Exercise 3
**A.** Plot three scatter plots of the data, where the x-axis of the scatter plot is an independent variable and the y-axis is the dependent variable.

**B.** Perform a linear regression using the three indepedent variables age, bmi, and bp. Write the regression equation you estimated. Is an increase in age associated with an increase or decrease in disease progression?



**C**. Calculate the MSE and $R^2$ of the regression model.

## Logistic Regression

Let's now look at performing classification via logistic regression. We will use the breast cancer dataset to predict the probability of a tumor being malignant (y = 0) or benign (y = 1) based on tumor features extracted from medical imaging.

Recall that for the case where there is a single independent variable, the model for logistic regression is:

$p(y=1) = (1+exp[-(b_0 + b_1x)])^{-1}$

In [ ]:
X,y = sk.datasets.load_breast_cancer(return_X_y=True)
mean_radius = X[:,0]

We can fit a logistic regression model using the mean radius of the tumor as the independent variable. Additionally, we can calculate the **confusion matrix** which tabulates the accuracy of our model.

| | $y$ = 0 | $y$ = 1     |
| :---        |    :----:   |          ---: |
| **$\hat{y}$ = 0**     | True Negative      | False Negative   |
| **$\hat{y}$ = 1**   | False Positive       | True Positive     |

In a perfect world, the confusion matrix would be diagonal. However you typically have a trade off in the amount of false positives and false negatives. It is your job as a researcher to decide which is worse. In this example, having too many false positives would result in patients being referred to invasive and expensive procedures they do not need. Conversely, by having too many false negatives, patients with malignant tumors would not seek treatment and there would be an increase in deaths from cancer.


## Exercise 4
**A.** Perform a logistic regression on y using the mean radius as the independent variable. Is increased tumor mean radius associated with an increase or decrease in the probability of the tumor being malignant?

**B.** Make a scatter plot with the x-axis mean radius and y-axis class of the tumor. Additionally, plot the sigmoidal curve you obtain from the coefficients you estimated in A.

**C.** Calculate the confusion matrix (**hint**:confusion_matrix($y,\hat{y}$)). Is this an acceptable amount of false negatives and false positives to you?

### Testing and Training Sets

An important concept in machine learning (and statistics) is the notion of **overfitting**. This is when your model performs very well on your training data, but has terrible performance for out-of-sample data. One solution to detect this is to split your data into **training** and **testing** sets. 

First, estimate the weights of the model using the data in the training set. Then, use the training and testing data as inputs into the estimated model to produce predicted values. If the predictions on the training data are good, but poor on the testing data, your model is overfit. It is almost always the case that your model will perform worse on the testing data, however it should not be that much worse.

For this next exercise, we will split our data into training and testing sets and predict tumor class using all of the indepedent variables in the breast cancer dataset. We will remove a random third of the data to be reserved as the testing set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

### Exercise 5
**A.** Perform a logistic regression on the training data.

**B.** Calculate the confusion matrix for the training data.

**C.** Calculate the confusion matrix for the testing data. Do you think your model was overfitting? Why or why not?

## MNIST

The MNIST dataset contains 70,000 handwritten digits with a 28x28 pixel resolution. Each digit is a number 0-9. Our task to is build a feedforward ANN to predict the digit based on the handwriting.

In [ ]:
(x_train,y_train), (x_test,y_test) = keras.datasets.mnist.load_data()

By looking at the shape of the training and testing data, we see that there are 60,000 examples in our training set and 10,000 in our testing set.

In [ ]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

Below are three examples from the training data.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(15, 4),constrained_layout=True)
for i in range(3):
    axes[i].set_title(f'Digit:{y_train[i]}')
    axes[i].imshow(x_train[i],cmap='gray',origin='upper')

In order to build our network, we will reshape our 28x28 pixel handwritten digits into vectors with a length of 784.

In [ ]:
n_examples_train = x_train.shape[0]
n_features = np.prod(x_train.shape[1:])
X_train = x_train.reshape((n_examples_train,n_features))

n_examples_test = x_test.shape[0]
X_test = x_test.reshape((n_examples_test,n_features))

We will now build our feedforward ANN with two hidden layers and one output layer. 

For each of the hidden layers we will use a **rectified linear unit** as the neuron activiation functions. Each hidden layer will be densely connected (i.e. all-to-all connections) and have 200 neurons.

The output layer will have 10 neurons, with each neuron corresponding to one of the ten digits. Since this is a classification task, we will use the **softmax** activation function on the output layer. This will result in a probability of an input being one of the 10 outputs. The predicted digit of a handwritten input will be the output digit with the largest probability.

The loss function that we will minimize is the **sparse categorical crossentropy** which is commonly used for classification tasks. The **adam** optimizer is a kind of gradient descent with extra bells and whistles. We will also store the classification accuracy of the model as it trains.

In [ ]:
def build_model():
    model = Sequential()
    model.add(Dense(200,input_shape = (n_features,),activation = 'relu'))
    model.add(Dense(200,activation = 'relu'))
    model.add(Dense(10,activation = 'softmax'))
    
    model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
    return(model)

In [ ]:
model = build_model()
model.summary()

We can see above that this relatively simple ANN has nearly 200,000 weights that need to be estimated. This means that we are performing gradient descent in a ~200,000 dimensional space. This would normally be a huge computational burden since the gradient would need to be calculated at every step in a huge dimensional space.

One solution to ease computational complexity is to use **stochastic gradient descent** (SGD). In SGD, random samples of your data (batches) are used to calculate the gradient of the loss function at each SGD iteration. For our example, we will use a batch size of 60. This means that SGD will run for 1000 iterations when estimating the weights.

One full sampling through the data set is called an **epoch**. Obviously, by using just one epoch we are not assured that our SGD alogrithm has converged. By using mulitple epochs, SGD will get closer to convergence. We will use 3 epochs for this notebook, but in practice you may use tens or hundreds of epochs and/or set a precision value.

In [ ]:
model.fit(X_train,y_train,batch_size = 60, epochs = 3)

We see that the final accuracy of the model is very high, however we need to check for overfitting. We can use the **model.predict()** function on our testing data to obtain digit probabilities for each testing input. The predicted digit for the test data is found by finding the maximum probability for each input. The **model.evaluate()** function is another way to do this  automatically.

In [ ]:
y_hat_probs = model.predict(X_test)
y_hat = np.argmax(y_hat_probs,axis =1)
val_loss, val_acc = model.evaluate(X_test,y_test)

Let's look at the digits that the ANN got wrong on the testing data. We can also plot the confusion matrix and a digit misclassification histogram.

In [ ]:
error_indx = []
for i in range(len(y_test)):
    if y_hat[i] != y_test[i]:
        error_indx.append(i)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(15, 4),constrained_layout=True)

for i in range(3):
    axes[i].set_title(f'Predicted Digit:{y_hat[error_indx[i]]}, Actual Digit:{y_test[error_indx[i]]}')
    axes[i].imshow(x_test[error_indx[i]],cmap='gray',origin='upper')

In [ ]:
CM = confusion_matrix(y_test,y_hat)
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(10, 5),constrained_layout=True)

axes[0].imshow(CM,origin='upper')
axes[0].set_xlabel("Actual Digit")
axes[0].set_ylabel("Predicted Digit")
axes[0].set_title("Confusion Matrix")
axes[0].set_xticks(range(10))
axes[0].set_yticks(range(10))

axes[1].hist(y_test[error_indx])
axes[1].set_xlabel("Digit")
axes[1].set_ylabel("Freqency")
axes[1].set_title("Digit Misclassification")

## Exercise 6
**A.** Below, construct and run a feedforward ANN identical to the one above, but now add a third hidden layer before the output layer. This new layer should have 100 densely connected neurons with **exponential linear unit** activation functions. How many parameters are now in the model? Use this ANN to answer the rest of the questions in the notebook.

**B.** What is the accuracy of this new ANN on the testing data?

**C.** Plot the confusion matrix and digit misclassification histogram.

## Exercise 7
Below are the test images with additive Gaussian noise with a mean of 0 and variance of 80.

In [ ]:
x_test_noise = x_test+np.random.normal(0,80,size = x_test.shape)

fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(15, 4),constrained_layout=True)
for i in range(3):
    axes[i].set_title(f'Digit:{y_test[i]}')
    axes[i].imshow(x_test_noise[i],cmap='gray',origin='upper')

**A.** What is the accuracy of the model using these noisy inputs?

**B.** Plot the confusion matrix and digit misclassification histogram.

## Exercise 8

Below are the test digits flipped upside down.

In [ ]:
x_test_flipped = np.flip(x_test)[::-1]

fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(15, 4),constrained_layout=True)
for i in range(3):
    axes[i].set_title(f'Digit:{y_test[i]}')
    axes[i].imshow(x_test_flipped[i],cmap='gray',origin='upper')

**A.** What is the accuracy of the model with these flipped digits? Are there certain digits you think the model will still perform well on?

**B.** Plot the confusion matrix and the digit misclassification histogram. What does this tell you about how ANNs learn to identify digits vs. how we learn?

## Exercise 9
Below are the negatives of the test images. Since the digits are in grayscale, this means every pixel is a number between 0 (black) and 255 (white).

In [ ]:
x_test_neg = 255-x_test

fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(15, 4),constrained_layout=True)
for i in range(3):
    axes[i].set_title(f'Digit:{y_test[i]}')
    axes[i].imshow(x_test_neg[i],cmap='gray',origin='upper')

**A.** Make a prediction of the accuracy of the model with these negative images. In particular, do you think the accuracy will be better, worse, or the same as the accuracy on the flipped digits?

**B.** Find the accuracy of the model with the negative digits.

**C.** Plot the confusion matrix and misclassification histogram. Are there any patterns you can see that may explain the accuracy of this model?